In [1]:
import os
%pwd

'f:\\Data Scientist\\TextSummarizerProject\\research'

In [2]:
os.chdir("../")
%pwd

'f:\\Data Scientist\\TextSummarizerProject'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass (frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: str
    
    # tokenizer_name: Path

In [4]:
from textSummarizer.constants.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(self, config_filepath= CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation
        create_directories([config.root_dir])
        data_transformation_config = DataTransformationConfig(
            root_dir= Path(config.root_dir),
            data_path= Path(config.data_path),
            # tokenizer_name=Path(config.tokenizer_name)
            tokenizer_name=config.tokenizer_name
        )
        return data_transformation_config

In [6]:
import os
from textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

f:\Data Scientist\TextSummarizerProject\vir_env-3.12\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [7]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_name, use_fast=False)
    # def __init__(self, config: DataTransformationConfig):
    #     self.config = config
    #     tokenizer_name = str(self.config.tokenizer_name).replace("\\", "/")
    #     self.tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

    def convert_examples_to_features(self, example_batch):
        input_recordings = self.tokenizer(example_batch['dialogue'], max_length=1024, truncation=True, padding='max_length')

        with self.tokenizer.as_target_tokenizer():
            target_recordings = self.tokenizer(example_batch['summary'], max_length=128, truncation=True, padding='max_length')
        return {
            'input_ids': input_recordings['input_ids'],
            'attention_mask': input_recordings['attention_mask'],
            'labels': target_recordings['input_ids']
        }
    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched=True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir, 'samsum_dataset'))

In [8]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    # data_transformation_config = DataTransformationConfig(
    #     root_dir=data_transformation_config.root_dir,
    #     data_path=data_transformation_config.data_path,
    #     tokenizer_name="google/pegasus-cnn_dailymail"
    # )
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise

[2025-06-27 07:27:07,399:INFO:common:yaml file: config\config.yaml loaded successfully.]
[2025-06-27 07:27:07,402:INFO:common:yaml file: params.yaml loaded successfully.]
[2025-06-27 07:27:07,406:INFO:common:created directory at: artifacts]
[2025-06-27 07:27:07,407:INFO:common:created directory at: artifacts/data_transformation]


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]f:\Data Scientist\TextSummarizerProject\vir_env-3.12\Lib\site-packages\transformers\tokenization_utils_base.py:3951: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 39551.12 examples/s]
